In [1]:
"""Import Libs"""
import requests
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

#------- Extract -------
"""Initializing Spark Session"""
spark = SparkSession.builder.appName("test").getOrCreate()
sc = spark.sparkContext
"""Read API That Shows University Information"""
r = requests.request("GET",'http://universities.hipolabs.com/search?country=United+States')
"""Print JSON Fetched file as python dictionary"""
data_j = r.json()

#-------- Transform------
"""Place a list of JSON Objects into a spark dataframe"""
df = sc.parallelize(data_j).map(lambda x: json.dumps(x))
df = spark.read.json(df)
"""The File had a multitude of Null values in the state column which is what I wanted to work with"""
df = df.na.drop(subset=['state-province'])

"""Show Data-Set"""
print('Count of colleges:',df.count())
print(df.columns)
df.select('domains','name','state-province','web_pages').show(truncate=False)

"""Count the number of universities that appeard in the list per state"""
state_sum = df.groupBy("state-province").count()
state_sum.show(50)

"""University Name Contains the name of the state"""
state_univers = df.filter(col("name").contains(col("state-province")))
print('State Name is in College Name:',state_univers.count())
state_univers.select('state-province','name').show(state_univers.count(),truncate=False)

Count of colleges: 83
['alpha_two_code', 'country', 'domains', 'name', 'state-province', 'web_pages']
+------------------------------------------+-------------------------------------------------+--------------+-----------------------------------------+
|domains                                   |name                                             |state-province|web_pages                                |
+------------------------------------------+-------------------------------------------------+--------------+-----------------------------------------+
|[upmc.edu, upmc.com]                      |University of Pittsburgh Medical Center          |Pennsylvania  |[https://www.upmc.com/]                  |
|[utrgv.edu]                               |The University of Texas at Rio Grande Valley     |Texas         |[https://www.utrgv.edu/]                 |
|[uvu.edu]                                 |Utah Valley University                           |Utah          |[https://uvu.edu/]           

In [2]:
#------- Load -------
import sqlite3
from sqlite3 import Error

conn = sqlite3.connect('University.db',timeout=10)

cur = conn.cursor()

cur.execute('''CREATE TABLE UNIVERSITY ( 
            STATE VARCHAR(60),
            UNIVERSITY_NAME VARCHAR(60))''')

Key = 1
for row in df.rdd.collect():
    conn.execute("INSERT INTO UNIVERSITY VALUES(?,?)",(row[3],row[4]))
    Key += 1

Cursor = conn.execute("SELECT * FROM UNIVERSITY")
for row in Cursor:
    print(row)

conn.close()

('University of Pittsburgh Medical Center', 'Pennsylvania')
('The University of Texas at Rio Grande Valley', 'Texas')
('Utah Valley University', 'Utah')
('College of Southern Nevada', 'NV')
('Bloomsburg University of Pennsylvania', 'Pennsylvania')
('California University of Pennsylvania', 'Pennsylvania')
('Clarion University of Pennsylvania', 'Pennsylvania')
('Allen College', 'Iowa')
('East Stroudsburg State University of Pennsylvania', 'Pennsylvania')
('Edinboro University of Pennsylvania', 'Pennsylvania')
('Emory & Henry College', 'VA')
('East Texas Baptist University', 'TX')
('Haverford College in Pennsylvania', 'Pennsylvania')
('Indiana University - Bloomington', 'Indiana')
('Indiana University of Pennsylvania', 'Pennsylvania')
('Kutztown University of Pennsylvania', 'Pennsylvania')
('Mansfield University of Pennsylvania', 'Pennsylvania')
('Mercyhurst University', 'Pennsylvania')
('Denver College of Nursing', 'Colorado')
('Millersville University of Pennsylvania', 'Pennsylvania')
(

In [3]:
df

DataFrame[alpha_two_code: string, country: string, domains: array<string>, name: string, state-province: string, web_pages: array<string>]

In [4]:
df.show()

+--------------+-------------+--------------------+--------------------+--------------+--------------------+
|alpha_two_code|      country|             domains|                name|state-province|           web_pages|
+--------------+-------------+--------------------+--------------------+--------------+--------------------+
|            US|United States|[upmc.edu, upmc.com]|University of Pit...|  Pennsylvania|[https://www.upmc...|
|            US|United States|         [utrgv.edu]|The University of...|         Texas|[https://www.utrg...|
|            US|United States|           [uvu.edu]|Utah Valley Unive...|          Utah|  [https://uvu.edu/]|
|            US|United States|           [csn.edu]|College of Southe...|            NV|[https://www.csn....|
|            US|United States|        [bloomu.edu]|Bloomsburg Univer...|  Pennsylvania|[http://www.bloom...|
|            US|United States|           [cup.edu]|California Univer...|  Pennsylvania|[http://www.cup.e...|
|            US|Uni